# Business Goals
A key activity of the banking sector is giving out loans. Every successful loan nets the bank a profit that corresponds to the interest it charged on the loan. On the other hand, when the loan is not successful, the bank loses a part or even the totality of the funds it had lent.

Therefore, the bank's business goal is, on the one hand, to understand how its client's characteristics affect the loans' success rate and, on the other, to be able to evaluate loan requests and decide if they should be approved or not, to make sure it lends to as many people who are able to pay them back (increasing revenue) and denies the loan to anyone who would not be paying it (decreasing costs).

# Data Mining Goals
We are using data mining to achieve the business goal previously enunciated. We identified the following goals to consider the technique's application to be a success:

## Descriptive Analysis
 * We are able to analyse the customer data across several axes, including balances, cards, transaction history, past loans, demographics, etc.
 * We are able to identify imbalances between the collected data (training set) and the real population (testing set) and understand if, and how, the data differs in cardinality and distribution.
 * We are able to identify problems with our data, including outliers and missing data, and correct them.
 * We are able to create knowledge that allows us to identify the trends and data features that are predictive of a loans viability.

## Prescriptive Analysis
 * We can build or train a computational model that can classify and predict whether a loan will be successful when it is presented with relevant data about the client.
 * The model has high precision - if a loan is predicted to be successful, it should have a high probability of being successful. (low rate of false positives).
 * The model has high recall - if a loan is successful, it should have a high probability of being predicted as successful. (low rate of false negatives).
 * The model is transparent - it should be possible to understand what factors lead a loan to be predicted as successful or otherwise, and what how confident is the model in that classification.

# Data Integration

We were given a dataset that consisted of a series of .csv files that represented, in the relational paradigm, information about the bank's clients and activities. The following relations are present:
 * Demographic indicators about the regions that the bank operates in;
 * Basic information about the bank's clients, in connection to one of the regions;
 * Basic information about the bank's accounts, in connection to one of the regions;
 * Information about cards issued by the bank;
 * Information (dispositions) that linked cards, the client to which the card was issued, and the account that is linked to the card;
 * Transaction histories, including balances after each transaction, for each account;
 * Information about past loans, including whether they were successful, and a set of unclassified loans whose success is to be predicted.

We normalized these .csv files using the `data_cleaner.py` script, and imported the files, using the `db_importer.py` script, to two SQLite 3 databases, namely `database_train.db` and `database_test.db`.

After all of this was done, we moved to integrate the different information sources and join them into a single tabular source of information. To do this, we used the SQL we have learned to join information across each relation, starting from the loans. Whenever we encountered cardinalities in that were not zero or one-to-one, we duplicated entries or aggregated them as we saw fit. At the moment, we are using the following query:

```sql
with districts_short as (
    select id, ratio_urban_inhabitants, avg_salary,
           coalesce(unemployment_96, unemployment_95) as unemployment,
           cast(coalesce(crimes_96, crimes_95) as real) / no_inhabitants as crimes from districts
), t as (
    select account_id, count(*) transaction_count, avg(amount) transaction_avg_amount, min(balance) transaction_min_balance,
           max(balance) transaction_max_balance, avg(balance) transaction_avg_balance
    from transactions
    group by account_id
)
select l.date loan_date, l.amount loan_amount, l.duration loan_duration , l.payments loan_payments,
       a.date account_creation, t.transaction_count, t.transaction_avg_amount, t.transaction_min_balance,
       t.transaction_max_balance, t.transaction_avg_balance,
       d.ratio_urban_inhabitants, d.avg_salary, d.unemployment, d.crimes,
       c2.birthday as client_birthday, l.status as loan_status
from loans l
    left join accounts a on l.account_id = a.id
    left join t on t.account_id = a.id
    left join districts_short d on a.district_id = d.id
left join dispositions d2 on a.id = d2.account_id
left join clients c2 on d2.client_id = c2.id;
```

You can see that we are collected a sizeable amount of features to start out, but we hoped to remove or combine features before we moved on to more involved methods of analysis.

In [2]:
import pandas as pd
import pandas.plotting as pdp

train_dataset = pd.read_csv("features/v1/learn.csv")